# EDA - preping the industry data

In [1]:
import pyspark 
sc = pyspark.sql.SparkSession.Builder()\
        .master('yarn')\
        .appName('hive-test')\
        .enableHiveSupport()\
        .getOrCreate()

In [2]:
sc

In [3]:
sc.sql('SHOW TABLES').collect()

[Row(database='default', tableName='pac_spending', isTemporary=False),
 Row(database='default', tableName='pac_spending_by_type', isTemporary=False),
 Row(database='default', tableName='pq_crp_cands18', isTemporary=False),
 Row(database='default', tableName='pq_crp_cmtes18', isTemporary=False),
 Row(database='default', tableName='pq_crp_expends18', isTemporary=False),
 Row(database='default', tableName='pq_crp_indivs18', isTemporary=False),
 Row(database='default', tableName='pq_crp_industry_names', isTemporary=False),
 Row(database='default', tableName='pq_crp_pac_other18', isTemporary=False),
 Row(database='default', tableName='pq_crp_pacs18', isTemporary=False),
 Row(database='default', tableName='pq_pp_bills', isTemporary=False),
 Row(database='default', tableName='pq_pp_sponsors', isTemporary=False),
 Row(database='default', tableName='pq_pp_titles', isTemporary=False),
 Row(database='default', tableName='pq_us_fec_ids', isTemporary=False),
 Row(database='default', tableName='pq_u

In [7]:
sc.sql('SELECT * FROM pq_crp_cands18').limit(10).show()

+-----+---------+---------+--------------------+-----+------------+----------+--------+---------+------+---------+------+
|cycle|feccandid|      cid|          firstlastp|party|distidrunfor|distidcurr|currcand|cyclecand|crpico|recipcode|nopacs|
+-----+---------+---------+--------------------+-----+------------+----------+--------+---------+------+---------+------+
| 2018|H6FL08221|N00028542|       D J Mauro (I)|    I|        FL08|          |        |         |      |       3N|      |
| 2018|H6NC09200|N00035491|     Mark Harris (R)|    R|        NC09|          |        |         |     O|       RO|      |
| 2018|H4NE01163|N00035619| Dennis Crawford (D)|    D|        NE01|          |        |        Y|     C|       DL|      |
| 2018|H0IN01127|N00041294| Arturas Kerelis (D)|    D|        IN01|          |        |         |      |       DN|      |
| 2018|H8AL02171|N00041295|     Barry Moore (R)|    R|        AL02|          |        |        Y|     C|       RL|      |
| 2018|H8AL04078|N000412

Does the individual table not have industry??

In [48]:
sc.sql('SELECT * FROM pq_crp_indivs18').limit(5).show()

+-----+-------------------+------------+-------------------+---------+-------+------+--------+----------+------+------+---------+-----+-----+---------+----+---------+---------+------+------------------+------------+------------+------+
|cycle|         fectransid|   contribid|            contrib|  recipid|orgname|ultorg|realcode|      date|amount|street|     city|state|  zip|recipcode|type|   cmteid|  otherid|gender|         microfilm|  occupation|    employer|source|
+-----+-------------------+------------+-------------------+---------+-------+------+--------+----------+------+------+---------+-----+-----+---------+----+---------+---------+------+------------------+------------+------------+------+
| 2018|4021520181504779816|p0003541768 |WUTHRICH, STEPHANIE|C00603084|       |      |   J1200|2017-11-08|     5|      |  ELKHART|   IN|46514|       PI| 15E|C00603084|C00401224|     F|201801319091098155|NOT EMPLOYED|        NONE| P/PAC|
| 2018|4021520181504779818|p0003541768 |WUTHRICH, STEPHA

In [51]:
sc.sql('SELECT realcode, sum(amount), count(contribid) FROM pq_crp_indivs18 GROUP BY realcode ').count()

470

In [55]:
sc.sql('SELECT realcode, sum(amount) AS money, count(contribid) FROM pq_crp_indivs18 GROUP BY realcode ORDER BY money DESC').collect()

[Row(realcode='Z9500', money=307821522, count(contribid)=2563962),
 Row(realcode='Y4000', money=227928460, count(contribid)=571205),
 Row(realcode='X1200', money=186889529, count(contribid)=972393),
 Row(realcode='K1000', money=94909387, count(contribid)=161371),
 Row(realcode='F2700', money=65099756, count(contribid)=4112),
 Row(realcode='F2600', money=57948077, count(contribid)=14944),
 Row(realcode='J1200', money=55168158, count(contribid)=1102926),
 Row(realcode='Y1000', money=49244720, count(contribid)=279555),
 Row(realcode='F2100', money=49068817, count(contribid)=48934),
 Row(realcode='Z9000', money=48915983, count(contribid)=10148),
 Row(realcode='F4000', money=46599518, count(contribid)=39289),
 Row(realcode='J1100', money=40848862, count(contribid)=71784),
 Row(realcode='F4100', money=38014293, count(contribid)=15586),
 Row(realcode='LB100', money=34695187, count(contribid)=208341),
 Row(realcode='JE300', money=33903445, count(contribid)=8861),
 Row(realcode='K2000', money=3

^Looks like all individuals have an industry, which is impressive given the number of blanks. Maybe one of the codes is unknown. Haven't found a translation table yet for realcodes 

In [ ]:
criteria for filter individuals:
    
Realcode not like Z9* (these are noncontributions)
Limit types to 10, 11, 15, 15E, 15J, 22Y
Remember that Type 10 is soft money before 2004 and Levin Funds or outside spending 2004+
RecipID like N* limits to candidates
Source <> P/PAC excludes contributions to PACs other than leadership PACs

****MUST COME BACK TO AND JOIN WITH PAC CANDIDATE CONTRIBUTIONS****

In [42]:
sc.sql('SELECT * FROM pq_crp_cmtes18').limit(10).show()

+-----+---------+--------------------+--------------------+--------------------+---------+---------+---------+-----+--------+------+---------+-------+------+
|cycle|   cmteid|            pacshort|           affiliate|              ultorg|  recipid|recipcode|feccandid|party|primcode|source|sensitive|foreign|active|
+-----+---------+--------------------+--------------------+--------------------+---------+---------+---------+-----+--------+------+---------+-------+------+
| 2018|C00000018|                    |                    |                    |C00000018|         |H8TX22313|     |        |      |         |  false|     0|
| 2018|C00000059|      Hallmark Cards|                    |      Hallmark Cards|C00000059|       PB|         |     |   C1400| WebAM|        n|  false|     1|
| 2018|C00000422|American Medical ...|American Medical ...|American Medical ...|C00000422|       PB|         |     |   H1100| AFP88|        n|  false|     1|
| 2018|C00000489| Teamsters Local 886|     Teamsters

In [12]:
sc.sql('SELECT primcode FROM pq_crp_cmtes18 GROUP BY primcode').count()

408

In [15]:
sc.sql('SELECT primcode FROM pq_crp_cmtes18 GROUP BY primcode').show(10)

+--------+
|primcode|
+--------+
|   C2100|
|   C1100|
|   T3100|
|   L1300|
|   T9000|
|   j7000|
|   J9000|
|   J1000|
|   M2400|
|   G6700|
+--------+
only showing top 10 rows



In [16]:
sc.sql('SELECT * FROM pq_crp_expends18').limit(1).show()

+-----+-------+-------------+----------+---------+--------+--------------------+-------+------+----------+----------+-----+---------+---------+------+----+--------------------+-----+---------+-------+------+
|Cycle|     ID|      TransID|CRPFilerid|Recipcode|Pacshort|        CRPRecipname|Expcode|Amount|      Date|      City|State|      Zip|CmteID_EF|Candid|Type|             Descrip|   PG|ElecOther|EntType|Source|
+-----+-------+-------------+----------+---------+--------+--------------------+-------+------+----------+----------+-----+---------+---------+------+----+--------------------+-----+---------+-------+------+
| 2018|8500179|SB23_86287240| C00401224|       PI| ActBlue|Progressive Turno...|    R50|   5.0|2017-07-05|NORTHBROOK|   IL|60065    |C00580068|      |    |Earmark Contribution|P2016|         |    PAC| @new2|
+-----+-------+-------------+----------+---------+--------+--------------------+-------+------+----------+----------+-----+---------+---------+------+----+-------------

In [18]:
sc.sql('SELECT * FROM pq_crp_pacs18').limit(10).show()

+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
|cycle|           fecrecno|    pacid|      cid|amount|      date|realcode|type| di|feccandid|
+-----+-------------------+---------+---------+------+----------+--------+----+---+---------+
| 2018|4031520181518958016|C00150797|N00035527|1000.0|2018-02-06|   K1200| 24K|  D|H4AR04048|
| 2018|4021520181504727622|C00343459|N00028152|5000.0|2017-12-27|   H1130| 24K|  D|H6CA22125|
| 2018|4062120181570992000|C00005249|N00035509|3000.0|2018-05-18|   F3300| 24K|  D|H4IA03115|
| 2018|4121120171484669562|C00449165|N00003689|1000.0|2017-11-08|   M3100| 24K|  D|H8OH01043|
| 2018|4101220171458221652|C00171843|N00002408|5000.0|2017-09-21|   F3400| 24K|  D|H2SC02042|
| 2018|4051820181565917536|C00035675|N00031933|1000.0|2018-04-23|   H4300| 24K|  D|H0MA10082|
| 2018|4072420171429498467|C00035451|N00034584|2000.0|2017-06-27|   LT100| 24K|  D|S2ME00158|
| 2018|4112020171465723593|C00033779|N00035346|1000.0|2017-1

In [20]:
sc.sql('SELECT * FROM pq_crp_pacs18').count()

191318

In [ ]:
need to joing the  pq_crp_pacs18 to the candidate table to get party detail. I think realcode and primcode are the same thing...
-To calculate direct contributions to candidates, you limit to DI = D

In [28]:
sc.sql('SELECT sum(amount) FROM pq_crp_pacs18 where di == "D"').limit(10).show()

+------------+
| sum(amount)|
+------------+
|3.21276746E8|
+------------+



In [ ]:
Need industry names!

In [113]:
import re
import pandas as pd
#crp_industry_names = ast.literal_eval(open("crp_industry_names.txt", "r").read
                                      
crp_industry_names = open("crp_industry_names.txt", "r").read()#.split('\t')
crp_industry_names
crp_industry_names = crp_industry_names.splitlines()


crp_industry_names = [re.split(r'\t+',x) for x in crp_industry_names]
crp_industry_names =pd.DataFrame(crp_industry_names)

crp_industry_names.columns=("Catcode","Catname","Catorder","Industry","Sector","Sector Long")
crp_industry_names= crp_industry_names[1:]
crp_industry_names.head()

,Catcode,Catname,Catorder,Industry,Sector,Sector Long
1,A0000,Agriculture,A11,Misc Agriculture,Agribusiness,Agribusiness
2,A1000,Crop production & basic processing,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
3,A1100,Cotton,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
4,A1200,Sugar cane & sugar beets,A01,Crop Production & Basic Processing,Agribusiness,Agribusiness
5,A1300,Tobacco & Tobacco products,A02,Tobacco,Agribusiness,Agribusiness


In [9]:
sc.sql('SELECT sector FROM pq_crp_industry_names GROUP BY Sector').show()

+--------------------+
|              sector|
+--------------------+
|               Labor|
|          Party Cmte|
|Finance/Insur/Rea...|
|Joint Candidate C...|
|Communic/Electronics|
| Lawyers & Lobbyists|
|Ideology/Single-I...|
|    Non-contribution|
|              Health|
|             Unknown|
|       Misc Business|
|               Other|
|        Construction|
|           Candidate|
|        Agribusiness|
| Energy/Nat Resource|
|             Defense|
|      Transportation|
+--------------------+



In [11]:
sc.sql('SELECT industry FROM pq_crp_industry_names GROUP BY industry').count()

103

In [39]:
sc.sql('SELECT * FROM pq_us_legislators').limit(1).show()

+---------+----------+-----------+------+--------+---------+----------+------+----+-----+--------+------------+-------------------+---+-------+-----+------------+-------+-------+--------+-------+----------+-----------+---------+--------------+------+--------+-----------+------------+--------------+------------------+--------+--------------------+
|last_name|first_name|middle_name|suffix|nickname|full_name|  birthday|gender|type|state|district|senate_class|              party|url|address|phone|contact_form|rss_url|twitter|facebook|youtube|youtube_id|bioguide_id|thomas_id|opensecrets_id|lis_id|cspan_id|govtrack_id|votesmart_id|ballotpedia_id|washington_post_id|icpsr_id|        wikipedia_id|
+---------+----------+-----------+------+--------+---------+----------+------+----+-----+--------+------------+-------------------+---+-------+-----+------------+-------+-------+--------+-------+----------+-----------+---------+--------------+------+--------+-----------+------------+--------------+---

In [43]:
pq_us_legislators.dtypes

NameError: name 'pq_us_legislators' is not defined

In [46]:
sc.sql('SELECT pacid,t2.cid, amount, date, realcode,t1.type, di,t1.feccandid,firstlastp, t2.party, t3.Industry, t3.Sector, t4.bioguide_id  \
       FROM pq_crp_pacs18 t1 \
       INNER JOIN pq_crp_cands18 t2 \
       ON t1.feccandid= t2.feccandid \
       INNER JOIN pq_crp_industry_names t3 \
       ON t1.realcode = t3.Catcode \
       LEFT JOIN pq_us_legislators t4 \
       ON t2.cid = t4.opensecrets_id \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%" AND \
       realcode NOT LIKE "z4*%"').limit(10).show()

#realcode is industry, this filters out transfers and joint fund raising committees

+---------+---------+------+----------+--------+----+---+---------+--------------------+-----+--------------------+--------------------+-----------+
|    pacid|      cid|amount|      date|realcode|type| di|feccandid|          firstlastp|party|            Industry|              Sector|bioguide_id|
+---------+---------+------+----------+--------+----+---+---------+--------------------+-----+--------------------+--------------------+-----------+
|C00150797|N00035527|1000.0|2018-02-06|   K1200| 24K|  D|H4AR04048| Bruce Westerman (R)|    R|   Lawyers/Law Firms| Lawyers & Lobbyists|    W000821|
|C00343459|N00028152|5000.0|2017-12-27|   H1130| 24K|  D|H6CA22125|  Kevin McCarthy (R)|    R|Health Professionals|              Health|    M001165|
|C00005249|N00035509|3000.0|2018-05-18|   F3300| 24K|  D|H4IA03115|     David Young (R)|    R|           Insurance|Finance/Insur/Rea...|    Y000066|
|C00449165|N00003689|1000.0|2017-11-08|   M3100| 24K|  D|H8OH01043|    Steve Chabot (R)|    R|Misc Manufac

## Create Links

In [49]:
sc.sql('SELECT sum(amount) ,feccandid, bioguide_id, firstlastp, party, Industry \
FROM (SELECT pacid,t2.cid, amount, date, realcode, t1.type, di,t1.feccandid,firstlastp, t2.party, t3.Industry, t3.Sector, t4.bioguide_id \
       FROM pq_crp_pacs18 t1 \
       INNER JOIN pq_crp_cands18 t2 \
       ON t1.feccandid= t2.feccandid \
       INNER JOIN pq_crp_industry_names t3 \
       ON t1.realcode = t3.Catcode \
       LEFT JOIN pq_us_legislators t4 \
       ON t1.cid = t4.opensecrets_id \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%" AND \
       realcode NOT LIKE "z4*%") sub \
       GROUP BY feccandid,firstlastp,bioguide_id, party, Industry ').limit(10).show()

+-----------+---------+-----------+------------------+-----+--------------------+
|sum(amount)|feccandid|bioguide_id|        firstlastp|party|            Industry|
+-----------+---------+-----------+------------------+-----+--------------------+
|    66500.0|H6TX02079|    B001291|   Brian Babin (R)|    R|       Air Transport|
|    20000.0|H4WI04183|    M001160|    Gwen Moore (D)|    D|    Commercial Banks|
|    16000.0|H8OH05036|    L000566|Robert E Latta (R)|    R|Chemical & Relate...|
|   117100.0|S6UT00063|    H000338| Orrin G Hatch (R)|    R|Pharmaceuticals/H...|
|    12000.0|H6AK00045|    Y000033|     Don Young (R)|    R|Crop Production &...|
|    35250.0|H2CA08164|    C001094|     Paul Cook (R)|    R|Building Material...|
|    14000.0|H0VA02118|    T000477|Scott W Taylor (R)|    R|           Oil & Gas|
|     2000.0|H2OH17109|    R000577|      Tim Ryan (D)|    D|Health Services/HMOs|
|    11000.0|H8NY29032|    C001092| Chris Collins (R)|    R|               Dairy|
|     8500.0|H4M

In [51]:
industry_viz = (sc.sql('SELECT sum(amount) ,feccandid, bioguide_id, firstlastp, party, Industry \
FROM (SELECT pacid,t2.cid, amount, date, realcode, t1.type, di,t1.feccandid,firstlastp, t2.party, t3.Industry, t3.Sector, t4.bioguide_id \
       FROM pq_crp_pacs18 t1 \
       INNER JOIN pq_crp_cands18 t2 \
       ON t1.feccandid= t2.feccandid \
       INNER JOIN pq_crp_industry_names t3 \
       ON t1.realcode = t3.Catcode \
       LEFT JOIN pq_us_legislators t4 \
       ON t1.cid = t4.opensecrets_id \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%" AND \
       realcode NOT LIKE "z4*%") sub \
       GROUP BY feccandid,firstlastp,bioguide_id, party, Industry '))

In [61]:
sc.sql('SELECT sum(amount) as contribution ,feccandid, bioguide_id, firstlastp, party, Industry \
FROM (SELECT pacid,t2.cid, amount, date, realcode, t1.type, di,t1.feccandid,firstlastp, t2.party, t3.Industry, t3.Sector, t4.bioguide_id \
       FROM pq_crp_pacs18 t1 \
       INNER JOIN pq_crp_cands18 t2 \
       ON t1.feccandid= t2.feccandid \
       INNER JOIN pq_crp_industry_names t3 \
       ON t1.realcode = t3.Catcode \
       LEFT JOIN pq_us_legislators t4 \
       ON t1.cid = t4.opensecrets_id \
       WHERE di == "D" AND \
       realcode NOT LIKE "z9*%" AND \
       realcode NOT LIKE "z4*%") sub \
       GROUP BY feccandid,firstlastp,bioguide_id, party, Industry ').flatMap(lambda row: {"source": row['Industry'], "target": row['feccandid'], "value": row['contribution']}).collect()

AttributeError: 'DataFrame' object has no attribute 'flatMap'

In [24]:
type(industry_viz)

pyspark.sql.dataframe.DataFrame

In [25]:
from pyspark.sql import HiveContext
sqlContext= HiveContext(sc)

industry_viz_data = sqlContext.createDataFrame(industry_viz)

TypeError: data is already a DataFrame

In [30]:
industry_viz[1]

Column<b'cid'>

Transforming into format needed for network viz

In [33]:
sc.sql('SELECT amount ,feccandid,firstlastp, party, Industry \
       FROM industry_viz').limit(10)

AnalysisException: 'Table or view not found: industry_viz; line 1 pos 65'

In [ ]:
grouped_src_dst = src_dst.groupby(['CMTE_ID', 'CAND_ID','STATE'])['TRANSACTION_AMT'].sum().reset_index()
#grouped_src_dst['TRANSACTION_AMT'].agg(np.sum)
grouped_src_dst.head()

In [63]:
industry_viz_data = industry_viz.toPandas()

In [64]:
industry_viz_data.head()

,sum(amount),feccandid,bioguide_id,firstlastp,party,Industry
0,66500.0,H6TX02079,B001291,Brian Babin (R),R,Air Transport
1,20000.0,H4WI04183,M001160,Gwen Moore (D),D,Commercial Banks
2,16000.0,H8OH05036,L000566,Robert E Latta (R),R,Chemical & Related Manufacturing
3,117100.0,S6UT00063,H000338,Orrin G Hatch (R),R,Pharmaceuticals/Health Products
4,12000.0,H6AK00045,Y000033,Don Young (R),R,Crop Production & Basic Processing


In [68]:
links_list = list(industry_viz_data.apply(lambda row: {"source": row['Industry'], "target": row['firstlastp'], "value": row['sum(amount)']}, axis=1))

In [69]:
links_list[1:10]

[{'source': 'Commercial Banks', 'target': 'Gwen Moore (D)', 'value': 20000.0},
 {'source': 'Chemical & Related Manufacturing',
  'target': 'Robert E Latta (R)',
  'value': 16000.0},
 {'source': 'Pharmaceuticals/Health Products',
  'target': 'Orrin G Hatch (R)',
  'value': 117100.0},
 {'source': 'Crop Production & Basic Processing',
  'target': 'Don Young (R)',
  'value': 12000.0},
 {'source': 'Building Materials & Equipment',
  'target': 'Paul Cook (R)',
  'value': 35250.0},
 {'source': 'Oil & Gas', 'target': 'Scott W Taylor (R)', 'value': 14000.0},
 {'source': 'Health Services/HMOs', 'target': 'Tim Ryan (D)', 'value': 2000.0},
 {'source': 'Dairy', 'target': 'Chris Collins (R)', 'value': 11000.0},
 {'source': 'Misc Manufacturing & Distributing',
  'target': 'Seth Moulton (D)',
  'value': 8500.0}]

## Making Nodes

In [96]:
import pandas as pd
df_nodes_I = pd.DataFrame(industry_viz_data.Industry.unique())
df_nodes_I["party"] = "Industry"
df_nodes_I['bioguide_id'] = 'flower'
df_nodes_I.columns = ['firstlastp' if x== 0 else x for x in df_nodes_P.columns]
df_nodes_I.head()

,firstlastp,party,bioguide_id
0,Air Transport,Industry,flower
1,Commercial Banks,Industry,flower
2,Chemical & Related Manufacturing,Industry,flower
3,Pharmaceuticals/Health Products,Industry,flower
4,Crop Production & Basic Processing,Industry,flower


In [97]:
df_nodes_P = industry_viz_data[['firstlastp', 'party', 'bioguide_id']]
df_nodes_P.head()

,firstlastp,party,bioguide_id
0,Brian Babin (R),R,B001291
1,Gwen Moore (D),D,M001160
2,Robert E Latta (R),R,L000566
3,Orrin G Hatch (R),R,H000338
4,Don Young (R),R,Y000033


In [103]:
df_nodes_P['bioguide_id'].unique()

array(['B001291', 'M001160', 'L000566', 'H000338', 'Y000033', 'C001094',
       'T000477', 'R000577', 'C001092', 'M001196', 'T000250', 'P000592',
       'M001199', 'C001112', 'G000579', 'G000583', 'C001053', 'R000570',
       'A000373', 'I000055', 'N000015', 'R000395', 'S001187', 'S001196',
       'C001077', 'W000795', 'G000551', 'C001035', 'L000551', 'W000820',
       'R000601', 'R000307', 'M000312', 'R000591', 'K000384', 'H001074',
       'C001063', 'K000188', 'D000612', 'H001046', 'P000595', 'W000437',
       'H001068', 'M001170', 'S001194', 'R000486', 'F000464', 'R000595',
       'D000096', 'S001198', 'F000459', 'R000575', 'B001298', 'R000576',
       'J000299', 'H001077', 'S001190', 'R000487', 'G000555', 'F000062',
       'B001243', 'B001227', 'M001158', 'D000607', 'B001287', 'P000611',
       'C001068', 'S001199', 'P000604', 'G000386', 'S001197', 'R000593',
       'J000297', 'K000383', 'K000379', 'J000298', 'M001189', 'M001159',
       'E000297', 'B001236', 'G000359', 'S001176', 

In [98]:
df_nodes = pd.concat([df_nodes_P,df_nodes_I])
df_nodes.head()

,firstlastp,party,bioguide_id
0,Brian Babin (R),R,B001291
1,Gwen Moore (D),D,M001160
2,Robert E Latta (R),R,L000566
3,Orrin G Hatch (R),R,H000338
4,Don Young (R),R,Y000033


In [101]:
df_nodes.shape

(27818, 3)

In [104]:
node_list = list(df_nodes.apply(lambda row: {"name": row['firstlastp'], "group": row['party'], "pic_id": row['bioguide_id']}, axis=1))

In [106]:
len(node_list)
node_list[-5:]

[{'name': 'Misc Agriculture', 'group': 'Industry', 'pic_id': 'flower'},
 {'name': 'Marijuana', 'group': 'Industry', 'pic_id': 'flower'},
 {'name': 'Joint Candidate Cmte', 'group': 'Industry', 'pic_id': 'flower'},
 {'name': 'Civil Servants/Public Officials',
  'group': 'Industry',
  'pic_id': 'flower'},
 {'name': 'Other', 'group': 'Industry', 'pic_id': 'flower'}]

### Merging Nodes and Links and exporting

In [107]:
json_prep = {"nodes":node_list, "links":links_list}
json_prep.keys()

dict_keys(['nodes', 'links'])

In [110]:
import json
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)
filename_out = 'industry_try1.json'
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()